# Search with base models

## Goal

Can we solve ARC tasks using base models with access to a DSL?

## First steps

In [ ]:
import importlib
import inspect

def extract_footprint(module_name: str, show_types: bool = False) -> str:
    """
    Load a module by name, then return a newline-separated list of all
    top-level functions in it, in the form:

      def func_name(arg1, arg2) -> return

    If show_types=True, annotations are included; otherwise only names.
    """
    mod = importlib.import_module(module_name)
    footprints = []

    for name, fn in inspect.getmembers(mod, inspect.isfunction):
        # skip imports from elsewhere
        if fn.__module__ != module_name or name.startswith("_"):
            continue

        sig = inspect.signature(fn)
        if not show_types:
            # strip type info
            params = [p.name for p in sig.parameters.values()]
            sig_text = f"({', '.join(params)})"
        else:
            sig_text = str(sig)

        footprints.append(f"- {name}{sig_text}")

    return "\n".join(footprints)

print(extract_footprint('arc25.BARC_dsl', show_types=True))

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/gbarbadillo/models/Qwen2.5-Coder-0.5B-Instruct")
len(tokenizer.tokenize(extract_footprint('arc25.BARC_dsl', show_types=True)))

2200 tokens just to teach how many DSL functions are available.

## TODO

- [ ] Create a prompt with the available DSL functions and the training ARC task
- [ ] Verify the effect of caching
- [ ] Update the library to be able to select which DSL to use when executing code
- [ ] Try to solve some easy task
- [ ] Create a refine prompt
- [ ] Make a more complex tree search